In [ ]:
# ! unzip -oq /home/aistudio/data/data85133/常规赛：PALM病理性近视预测.zip
# ! rm -rf __MACOSX
# ! mv 常规赛：PALM病理性近视预测 PLAM

In [ ]:
# !git clone https://gitee.com/paddlepaddle/PaddleClas.git -b release/2.0

In [ ]:
!mkdir pretrained
%cd /home/aistudio/pretrained
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ResNet50_vd_pretrained.pdparams
!wget https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/ResNet50_vd_ssld_pretrained.pdparams

###  创建train.txt  和 val.txt

In [ ]:
import os
import pandas as pd
import random
random.seed(2020)
class_xls='/home/aistudio/PLAM/Train/Classification.xlsx'
data_path='/home/aistudio/PLAM/Train/fundus_image'

train_f = open('/home/aistudio/PLAM/Train/train.txt','w') 
val_f = open('/home/aistudio/PLAM/Train/val.txt' ,'w')

name_label = pd.read_excel(class_xls).values
ratio = int(0.85 * len(name_label))
random.shuffle(name_label)
for i ,data in enumerate(name_label):
    name , label = data
    data_path = os.path.join(data_path, name)
    content = data_path  + ' ' + str(label) + '\n'
    print(content)
    if i < ratio :
        train_f.write(content)
    else:
        val_f.write(content)

train_f.close()
val_f.close()

In [ ]:
import os
import random
contents = list()
with open('/home/aistudio/PLAM/Train/train.txt', 'r') as f:
    for line in f.readlines():
        contents.append(line.strip())
with open('/home/aistudio/PLAM/Train/val.txt', 'r') as f:
    for line in f.readlines():
        contents.append(line.strip())
random.seed(2021)
random.shuffle(contents)

rate = int(len(contents) * 0.8)#训练集和测试集8：2
train_txt = open('/home/aistudio/PLAM/Train/train_new.txt','w')
val_txt = open('/home/aistudio/PLAM/Train/val_new.txt','w')
for i,line in enumerate(contents):
    if i < rate:
        train_txt.write(line +'\n')
    else:
        val_txt.write(line +'\n')
train_txt.close()
val_txt.close()
print('完成')

完成


In [ ]:
%cd /home/aistudio/PaddleClas
!python3 tools/train.py -c ./configs/quick_start/ResNet50_vd_ssld_random_erasing_finetune.yaml

In [ ]:
!python tools/eval.py \
    -c ./configs/quick_start/ResNet50_vd_ssld_random_erasing_finetune.yaml \
    -o pretrained_model="./output/ResNet50_vd/best_model/ppcls"\
    -o load_static_weights=False

### 导出模型

In [5]:
!python tools/export_model.py \
    --model ResNet50_vd \
    --pretrained_model ./output/ResNet50_vd/best_model/ppcls \
    --output_path ./inference \
    --class_dim=2

### 预测结果生成Classification_Results.csv


In [ ]:
!python tools/infer/predict.py \
    -i '/home/aistudio/PLAM/PALM-Testing400-Images/' \
    --model_file "./inference/inference.pdmodel" \
    --params_file "./inference/inference.pdiparams" \
    --use_gpu=True \
    --use_tensorrt=False \
    --class_num=2